In [6]:
import numpy as np
import tensorflow as tf
import random as rn
import os
os.environ['PYTHONHASHSEED'] = '0'

np.random.seed(42)
rn.seed(12345)
from keras.layers import Convolution2D, BatchNormalization, MaxPooling2D, Dense, Input, Dropout, Flatten
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.optimizers import Adam, Adagrad, SGD, RMSprop, Adadelta, Adamax, Nadam, Ftrl
import time
import pandas as pd
from keras.callbacks import TensorBoard
import os
from sklearn.model_selection import train_test_split
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from scipy import signal
from scipy.io import wavfile
import random

SyntaxError: invalid syntax (pywrap_tensorflow_internal.py, line 114)

In [3]:
def Load_Wav_(WorkDir):
    Input_Files = []
    Source_Samples= []
    for d, dirs, files in os.walk(WorkDir):
        for file in files:
            Input_Files.append(file)
    GFile  = []
    for file in Input_Files:
            if file.endswith(".wav"):
                sample_rate, samples = wavfile.read(str(WorkDir)+ file)
                if sample_rate != 8000:
                    continue
                if max(abs(samples)) < 410:
                    continue
                if len(samples) < int(0.1 * sample_rate):
                    continue
                GFile.append(file)
#                 samples = Convert_To_06(samples)
                Source_Samples.append(samples)

    return  Source_Samples, GFile



def Add_Zero(specgram, TargetColumnNumber, StartSignalPosition):

    if len(specgram[0]) >= TargetColumnNumber:
         return specgram

    full_array = np.zeros((len(specgram), TargetColumnNumber))
    full_array[:, :len(specgram[0])-TargetColumnNumber] = specgram
    if StartSignalPosition == 0:
        full_array[:, :len(specgram[0]) - TargetColumnNumber] = specgram
    elif StartSignalPosition == TargetColumnNumber - len(specgram[0]):
        full_array[:, StartSignalPosition:] = specgram
    else:
        full_array[:, StartSignalPosition:StartSignalPosition+len(specgram[0]) - TargetColumnNumber] = specgram

    return full_array
def log_specgram(audio, window_size, sample_rate=8000,
                 eps=1e-10, windoe_fuction='hann'):
    nperseg = int(round(window_size * sample_rate / 1000))
    noverlap = int(round(window_size / 2 * sample_rate / 1000))
    freqs, times, spec = signal.spectrogram(audio,
                                            fs=sample_rate,
                                            window=windoe_fuction,
                                            nperseg=nperseg,
                                            noverlap=noverlap,
                                            detrend=False)
    return freqs, times, np.log(spec.astype(np.float32) + eps)
def Convert_Wav_To_specgram(SamplesList, Input_Files, window_size, windoe_fuction):
    samples = np.zeros(int(0.6 * 8000))
    _, _, specgram = log_specgram(audio=samples, window_size=window_size, windoe_fuction=windoe_fuction)
    TargetColumnNumber = len(specgram[0])
    TargetRow = len(specgram)
    x = []
    y = []
    for i in range(len(SamplesList)):
        _, _, specgram = log_specgram(audio=SamplesList[i], window_size=window_size, windoe_fuction=windoe_fuction)
        specgram = Add_Zero(specgram, TargetColumnNumber, 0)
        x.append(specgram)
        file = Input_Files[i]
        if 'cl_1' in file:
            y.append([0, 0, 1])
        elif 'cl_2' in file:
            y.append([1, 0, 0])
        else:
            y.append([0, 1, 0])
        
 
    x = np.array(x)
    x = x.reshape(tuple(list(x.shape) + [1]))
    y = np.array(y)
    return x, y
def RandomozeArrays(SourceArrayX, SourceArrayY):
    TargetArrayX=[]
    TargetArrayY=[]
    while 0 < len(SourceArrayX):
        Index = random.randint(0, len(SourceArrayX) - 1)
        TargetArrayX.append(SourceArrayX[Index])
        del SourceArrayX[Index]
        TargetArrayY.append(SourceArrayY[Index])
        del SourceArrayY[Index]      
        
    return TargetArrayX,TargetArrayY

In [4]:
def Learn_NN_5L_(TrainDir,ValidDir, RezDir,NN_Name,Epochs=30, window_size=25, windoe_fuction='hann'):
    Source_Samples, Input_Files= Load_Wav_(TrainDir)
    print('end load train data', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
    Source_Samples, Input_Files = RandomozeArrays(Source_Samples, Input_Files)
    print('end Randomize train data', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
    
    
    X_Train, Y_Train = Convert_Wav_To_specgram(SamplesList=Source_Samples, Input_Files=Input_Files,
                                           window_size=window_size, windoe_fuction=windoe_fuction)
    print('end convert train data', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
     
    Source_Samples, Input_Files = Load_Wav_(ValidDir)
    print('end load valid data', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))

    X_val1, y_val1 = Convert_Wav_To_specgram(SamplesList=Source_Samples, Input_Files=Input_Files,
                                                     window_size=window_size, windoe_fuction=windoe_fuction)
    print('end convert valid data', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))

    input_shape = (X_Train.shape[1], X_Train.shape[2], 1)
    model = Sequential()

    model.add(BatchNormalization(input_shape = input_shape))
    model.add(Convolution2D(48, (5, 5), strides = (3, 3), padding = 'same',input_shape = input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding = 'same'))
    model.add(BatchNormalization())
    model.add(Convolution2D(40, (3, 3), strides = (2, 2), padding = 'same'))
    model.add(Activation('relu'))
    model.add(Flatten())
#     model.add(Dense(45))
#     model.add(Activation('relu'))
#     model.add(Dense(30))
#     model.add(Activation('relu'))
    model.add(Dense(15))
    model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(Activation('relu'))
    model.add(Dense(3))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    csv_logger = CSVLogger(RezDir+NN_Name+'_training__log.csv', separator=',', append=False)

    
    checkpoint = ModelCheckpoint(filepath=RezDir+NN_Name+'_Best.hdf5',
                 monitor='val_accuracy',
                 save_best_only=True,
                 mode='max',
                 verbose=1)
    model.fit(X_Train, Y_Train,
          batch_size = 64,
          epochs = Epochs,shuffle=True,
          validation_data=(X_val1, y_val1),
          callbacks=[checkpoint, csv_logger])
    model.save(filepath=RezDir+NN_Name+'_Final.hdf5')

In [5]:
def Learn_NN_5L_Custom(TrainDir,ValidDir, RezDir,NN_Name, Neurons=40, Epochs=30, window_size=25, windoe_fuction='hann'):
    Source_Samples, Input_Files= Load_Wav_(TrainDir)
    print('end load train data', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
    Source_Samples, Input_Files = RandomozeArrays(Source_Samples, Input_Files)
    print('end Randomize train data', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
    
    
    X_Train, Y_Train = Convert_Wav_To_specgram(SamplesList=Source_Samples, Input_Files=Input_Files,
                                           window_size=window_size, windoe_fuction=windoe_fuction)
    print('end convert train data', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
     
    Source_Samples, Input_Files = Load_Wav_(ValidDir)
    print('end load valid data', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))

    X_val1, y_val1 = Convert_Wav_To_specgram(SamplesList=Source_Samples, Input_Files=Input_Files,
                                                     window_size=window_size, windoe_fuction=windoe_fuction)
    print('end convert valid data', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))

    input_shape = (X_Train.shape[1], X_Train.shape[2], 1)
    model = Sequential()

    model.add(BatchNormalization(input_shape = input_shape))
    model.add(Convolution2D(48, (5, 5), strides = (3, 3), padding = 'same',input_shape = input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding = 'same'))
    model.add(BatchNormalization())
    model.add(Convolution2D(40, (3, 3), strides = (2, 2), padding = 'same'))
    model.add(Activation('relu'))
    model.add(Flatten())
#     model.add(Dense(45))
#     model.add(Activation('relu'))
#     model.add(Dense(15))
#     model.add(Activation('relu'))
#     model.add(Dense(10))
#     model.add(Activation('relu'))
    model.add(Dense(Neurons))
    model.add(Activation('relu'))
    model.add(Dense(3))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    csv_logger = CSVLogger(RezDir+NN_Name+'_training__log.csv', separator=',', append=False)

    
    checkpoint = ModelCheckpoint(filepath=RezDir+NN_Name+'_Best.hdf5',
                 monitor='val_accuracy',
                 save_best_only=True,
                 mode='max',
                 verbose=1)
    model.fit(X_Train, Y_Train,
          batch_size = 64,
          epochs = Epochs,shuffle=True,
          validation_data=(X_val1, y_val1),
          callbacks=[checkpoint, csv_logger])
    model.save(filepath=RezDir+NN_Name+'_Final.hdf5')

In [6]:
def TestNN_(NetName, SourceDir, TargetFile, window_size):
    Input_Files = []
    Source_Samples = []
    for d, dirs, files in os.walk(SourceDir):
        for file in files:
            if file.endswith(".wav"):
                sample_rate, samples = wavfile.read(SourceDir + file)
                if sample_rate != 8000:
                     continue
                if max(abs(samples)) < 410:
                    continue
                if len(samples) < int(0.1 * sample_rate):
                    continue
                Input_Files.append(file)
#                 samples = Convert_To_06(samples)
                Source_Samples.append(samples)

    x, y = Convert_Wav_To_specgram(SamplesList=Source_Samples, Input_Files=Input_Files,
                                             window_size=window_size, windoe_fuction='hann')

    new_model = load_model(NetName)
    pred = new_model.predict(x)
    f = open(TargetFile+'_FilesReport.csv', 'w', newline='\n')
    f.write('NetName = %s, Files %s \n'%(NetName,SourceDir))
    f.write('File Name,Marked As,Recognized As,Cl 1,Cl 2,Cl 3, \n')
    CodeList = ['Cl 2', 'Cl 3', 'Cl 1']
    SemplCount= [0,0,0]
    StatRez = [[0,0,0],[0,0,0],[0,0,0]]
    for i in range(len(pred)):
        YY = list(y[i])
        Rez = list(pred[i])
        TrueCalss = YY.index(max(YY))
        NNClass = Rez.index(max(Rez))
        SemplCount[TrueCalss] +=1
        StatRez[TrueCalss][NNClass] += 1
        f.write( '%s,%s,%s, %f , %f, %f,\n' % (Input_Files[i], CodeList[TrueCalss], CodeList[NNClass], pred[i][2], pred[i][0], pred[i][1]))

    f.close()
    f = open(TargetFile+'_Report.csv', 'w', newline='\n')
    f.write('NetName = %s, Files %s \n'%(NetName,SourceDir))
    f.write('Var,Cl 1,Cl 2,Cl 3, \n')

    f.write(  'Count,%s,%s, %s ,\n' % (SemplCount[2],SemplCount[0], SemplCount[1]))
    f.write('Cl 1 As,%s,%s, %s ,\n' % (StatRez[2][2], StatRez[2][0], StatRez[2][1]))
    f.write('Cl 2 As,%s,%s, %s ,\n' % (StatRez[0][2], StatRez[0][0], StatRez[0][1]))
    f.write('Cl 3 As,%s,%s, %s ,\n' % (StatRez[1][2], StatRez[1][0], StatRez[1][1]))
    trueclass =100.0* (StatRez[2][2] + StatRez[0][0] + StatRez[1][1])/ float( sum(SemplCount))
    
    for i in range(len(SemplCount)):
        for k in range(len(SemplCount)):
            if SemplCount[i] > 0:
                StatRez[i][k]=100.0*float(StatRez[i][k])/SemplCount[i]
    f.write('Cl 1 As %%,%.3f%%,%.3f%%, %.3f%% ,\n' % (StatRez[2][2], StatRez[2][0], StatRez[2][1]))
    f.write('Cl 2  As %%,%.3f%%,%.3f%%, %.3f%% ,\n' % (StatRez[0][2], StatRez[0][0], StatRez[0][1]))
    f.write('Cl 3 As %%,%.3f%%,%.3f%%, %.3f%% ,\n' % (StatRez[1][2], StatRez[1][0], StatRez[1][1]))
    
    f.write(',\n')
    
    f.write('Total acc. ,%.3f%% ,\n' % (trueclass))

    f.close()

In [7]:
def TestNN_Custom(NetName, SourceDir, TargetFolder, window_size):
    f = open(TargetFolder+'_Report.csv', 'a', newline='\n')
#     f.write('NetName = %s, Files %s, '%(NetName,SourceDir))
    f.write('NetName = %s'%(NetName))
    for dir in ('Train', 'Test', 'Valid'):
        Input_Files = []
        Source_Samples = []
        for d, dirs, files in os.walk(dir+"/"+SourceDir):
            for file in files:
                if file.endswith(".wav"):
                    sample_rate, samples = wavfile.read(dir+"/"+SourceDir + file)
                    if sample_rate != 8000:
                         continue
                    if max(abs(samples)) < 410:
                        continue
                    if len(samples) < int(0.1 * sample_rate):
                        continue
                    Input_Files.append(file)

                    Source_Samples.append(samples)

        x, y = Convert_Wav_To_specgram(SamplesList=Source_Samples, Input_Files=Input_Files,
                                                 window_size=window_size, windoe_fuction='hann')

        new_model = load_model(NetName)
        pred = new_model.predict(x)

        SemplCount= [0,0,0]
        StatRez = [[0,0,0],[0,0,0],[0,0,0]]
        for i in range(len(pred)):
            YY = list(y[i])
            Rez = list(pred[i])
            TrueCalss = YY.index(max(YY))
            NNClass = Rez.index(max(Rez))
            SemplCount[TrueCalss] +=1
            StatRez[TrueCalss][NNClass] += 1

        

        trueclass =100.0* (StatRez[2][2] + StatRez[0][0] + StatRez[1][1])/ float( sum(SemplCount))

        f.write('%.3f%%,' % (trueclass))
        
    f.write('\n')
    f.close()

In [30]:
folder=f"Results15Neurons"
folder = 'Lab4_experiment' + "/" + folder

print('---start Learn---', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
Learn_NN_5L_(TrainDir='/home/hlib/sem7/AI/NewData/Train/', 
             ValidDir='/home/hlib/sem7/AI/NewData/Valid/', 
             RezDir=f"/home/hlib/sem7/AI/{folder}/", 
             NN_Name='NN_L5', Epochs=25, window_size=25, windoe_fuction='hann')

print('---end  Learn---', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))

---start Learn--- 2022-11-13 12:01:40
end load train data 2022-11-13 12:01:58
end Randomize train data 2022-11-13 12:01:58
end convert train data 2022-11-13 12:02:00
end load valid data 2022-11-13 12:02:00
end convert valid data 2022-11-13 12:02:00
Epoch 1/25
102/102 [==============================] - ETA: 0s - loss: 0.6866 - accuracy: 0.6955
Epoch 1: val_accuracy improved from -inf to 0.75528, saving model to /home/hlib/sem7/AI/Lab4_experiment/Results15Neurons/NN_L5_Best.hdf5
102/102 [==============================] - 3s 27ms/step - loss: 0.6866 - accuracy: 0.6955 - val_loss: 0.7570 - val_accuracy: 0.7553
Epoch 2/25
100/102 [============================>.] - ETA: 0s - loss: 0.4076 - accuracy: 0.8316
Epoch 2: val_accuracy improved from 0.75528 to 0.78758, saving model to /home/hlib/sem7/AI/Lab4_experiment/Results15Neurons/NN_L5_Best.hdf5
102/102 [==============================] - 3s 26ms/step - loss: 0.4070 - accuracy: 0.8322 - val_loss: 0.5861 - val_accuracy: 0.7876
Epoch 3/25
100/102

In [31]:
for dir in ('Train', 'Test', 'Valid'):
    TestNN_(NetName=f"/home/hlib/sem7/AI/{folder}/NN_L5_Best.hdf5",
            SourceDir=f"/home/hlib/sem7/AI/NewData/{dir}/",
            TargetFile=f"/home/hlib/sem7/AI/{folder}/NN_L5_rez_{dir}",
            window_size=25)

26/26 [==============================] - 0s 5ms/step


In [8]:
def TestNN_Custom_2(NetName, SourceDir, TargetFile, window_size): 
    f = open(TargetFile+'Report.csv', 'a', newline='\n')
    
    Input_Files = []
    Source_Samples = []
    for d, dirs, files in os.walk(SourceDir):
        for file in files:
            if file.endswith(".wav"):
                sample_rate, samples = wavfile.read(SourceDir + file)
                if sample_rate != 8000:
                     continue
                if max(abs(samples)) < 410:
                    continue
                if len(samples) < int(0.1 * sample_rate):
                    continue
                Input_Files.append(file)
#           
                Source_Samples.append(samples)

    x, y = Convert_Wav_To_specgram(SamplesList=Source_Samples, Input_Files=Input_Files,
                                             window_size=window_size, windoe_fuction='hann')

    new_model = load_model(NetName)
    pred = new_model.predict(x)

    
    SemplCount= [0,0,0]
    StatRez = [[0,0,0],[0,0,0],[0,0,0]]
    for i in range(len(pred)):
        YY = list(y[i])
        Rez = list(pred[i])
        TrueCalss = YY.index(max(YY))
        NNClass = Rez.index(max(Rez))
        SemplCount[TrueCalss] +=1
        StatRez[TrueCalss][NNClass] += 1
   

   

 
    trueclass =100.0* (StatRez[2][2] + StatRez[0][0] + StatRez[1][1])/ float( sum(SemplCount))
    
    for i in range(len(SemplCount)):
        for k in range(len(SemplCount)):
            if SemplCount[i] > 0:
                StatRez[i][k]=100.0*float(StatRez[i][k])/SemplCount[i]

    f.write('%.3f%%,' % (trueclass))

    f.close()

In [32]:
folder=f"Results15Neurons"
folder = 'Lab4_experiment' + "/" + folder

print('---start Learn---', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
Learn_NN_5L_(TrainDir='/home/hlib/sem7/AI/NewData/Train/', 
             ValidDir='/home/hlib/sem7/AI/NewData/Valid/', 
             RezDir=f"/home/hlib/sem7/AI/{folder}/", 
             NN_Name='NN_L5', Epochs=25, window_size=25, windoe_fuction='hann')

print('---end  Learn---', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))

for dir in ('Train', 'Test', 'Valid'):
    TestNN_(NetName=f"/home/hlib/sem7/AI/{folder}/NN_L5_Best.hdf5",
            SourceDir=f"/home/hlib/sem7/AI/NewData/{dir}/",
            TargetFile=f"/home/hlib/sem7/AI/{folder}/NN_L5_rez_{dir}",
            window_size=25)

---start Learn--- 2022-11-13 12:11:21
end load train data 2022-11-13 12:11:23
end Randomize train data 2022-11-13 12:11:23
end convert train data 2022-11-13 12:11:25
end load valid data 2022-11-13 12:11:25
end convert valid data 2022-11-13 12:11:25
Epoch 1/25
100/102 [============================>.] - ETA: 0s - loss: 0.6746 - accuracy: 0.7000
Epoch 1: val_accuracy improved from -inf to 0.77640, saving model to /home/hlib/sem7/AI/Lab4_experiment/Results15Neurons/NN_L5_Best.hdf5
102/102 [==============================] - 3s 27ms/step - loss: 0.6713 - accuracy: 0.7018 - val_loss: 0.7021 - val_accuracy: 0.7764
Epoch 2/25
101/102 [============================>.] - ETA: 0s - loss: 0.4110 - accuracy: 0.8315
Epoch 2: val_accuracy improved from 0.77640 to 0.78012, saving model to /home/hlib/sem7/AI/Lab4_experiment/Results15Neurons/NN_L5_Best.hdf5
102/102 [==============================] - 3s 25ms/step - loss: 0.4117 - accuracy: 0.8310 - val_loss: 0.6112 - val_accuracy: 0.7801
Epoch 3/25
102/102

In [61]:
d = 'Lab4_experiment'

# for neuro in [15]: 
# for neuro in range (5, 201, 5): 
for neuro in range (205, 500, 5): 
    folder=f"Results{neuro}Neurons"
    folder = d + "/" + folder
    
    e = neuro // 15

    print(f"start train for {neuro} neurons")

    Learn_NN_5L_Custom(TrainDir='/home/hlib/sem7/AI/NewData/Train/', 
                 ValidDir='/home/hlib/sem7/AI/NewData/Valid/', 
                 RezDir=f"/home/hlib/sem7/AI/{folder}/", 
                 NN_Name='NN_L5', Neurons=neuro, Epochs=20+e, window_size=25, windoe_fuction='hann')


    NetName = f"/home/hlib/sem7/AI/{folder}/NN_L5_Best.hdf5"
    TargetFile = f"/home/hlib/sem7/AI/{d}/"

    f = open(TargetFile+'Report.csv', 'a', newline='\n')
    f.write('NetName = %s,'%(NetName))
    f.close()
    
    for dir in ('Train', 'Test', 'Valid'):
        TestNN_Custom_2(NetName=NetName,
                SourceDir=f"/home/hlib/sem7/AI/NewData/{dir}/",
                TargetFile=TargetFile,
                window_size=25)
        
    f = open(TargetFile+'Report.csv', 'a', newline='\n')
    f.write('\n')
    f.close()   
    
    print(f"finish train for {neuro} neurons")

start train for 205 neurons
end load train data 2022-11-13 14:53:18
end Randomize train data 2022-11-13 14:53:18
end convert train data 2022-11-13 14:53:19
end load valid data 2022-11-13 14:53:19
end convert valid data 2022-11-13 14:53:20
Epoch 1/33
101/102 [============================>.] - ETA: 0s - loss: 0.5974 - accuracy: 0.7463
Epoch 1: val_accuracy improved from -inf to 0.80497, saving model to /home/hlib/sem7/AI/Lab4_experiment/Results205Neurons/NN_L5_Best.hdf5
102/102 [==============================] - 3s 30ms/step - loss: 0.5959 - accuracy: 0.7470 - val_loss: 0.6325 - val_accuracy: 0.8050
Epoch 2/33
100/102 [============================>.] - ETA: 0s - loss: 0.3333 - accuracy: 0.8662
Epoch 2: val_accuracy improved from 0.80497 to 0.86584, saving model to /home/hlib/sem7/AI/Lab4_experiment/Results205Neurons/NN_L5_Best.hdf5
102/102 [==============================] - 3s 29ms/step - loss: 0.3322 - accuracy: 0.8670 - val_loss: 0.4191 - val_accuracy: 0.8658
Epoch 3/33
101/102 [======

Epoch 27/33
101/102 [============================>.] - ETA: 0s - loss: 1.7759e-04 - accuracy: 1.0000
Epoch 27: val_accuracy did not improve from 0.90683
102/102 [==============================] - 3s 30ms/step - loss: 1.7724e-04 - accuracy: 1.0000 - val_loss: 0.5044 - val_accuracy: 0.8969
Epoch 28/33
101/102 [============================>.] - ETA: 0s - loss: 1.5226e-04 - accuracy: 1.0000
Epoch 28: val_accuracy did not improve from 0.90683
102/102 [==============================] - 3s 30ms/step - loss: 1.5214e-04 - accuracy: 1.0000 - val_loss: 0.5023 - val_accuracy: 0.8969
Epoch 29/33
101/102 [============================>.] - ETA: 0s - loss: 1.4696e-04 - accuracy: 1.0000
Epoch 29: val_accuracy did not improve from 0.90683
102/102 [==============================] - 3s 30ms/step - loss: 1.4678e-04 - accuracy: 1.0000 - val_loss: 0.5153 - val_accuracy: 0.8969
Epoch 30/33
101/102 [============================>.] - ETA: 0s - loss: 1.2967e-04 - accuracy: 1.0000
Epoch 30: val_accuracy did not i

102/102 [==============================] - 3s 31ms/step - loss: 4.3474e-04 - accuracy: 1.0000 - val_loss: 0.4262 - val_accuracy: 0.9130
Epoch 19/34
101/102 [============================>.] - ETA: 0s - loss: 3.7928e-04 - accuracy: 1.0000
Epoch 19: val_accuracy did not improve from 0.91304
102/102 [==============================] - 3s 30ms/step - loss: 3.7932e-04 - accuracy: 1.0000 - val_loss: 0.4425 - val_accuracy: 0.9056
Epoch 20/34
100/102 [============================>.] - ETA: 0s - loss: 3.2946e-04 - accuracy: 1.0000
Epoch 20: val_accuracy did not improve from 0.91304
102/102 [==============================] - 3s 31ms/step - loss: 3.3176e-04 - accuracy: 1.0000 - val_loss: 0.4443 - val_accuracy: 0.9093
Epoch 21/34
100/102 [============================>.] - ETA: 0s - loss: 2.9867e-04 - accuracy: 1.0000
Epoch 21: val_accuracy did not improve from 0.91304
102/102 [==============================] - 3s 30ms/step - loss: 2.9835e-04 - accuracy: 1.0000 - val_loss: 0.4482 - val_accuracy: 0.91

102/102 [==============================] - 3s 31ms/step - loss: 0.3228 - accuracy: 0.8723 - val_loss: 0.4806 - val_accuracy: 0.8311
Epoch 3/34
101/102 [============================>.] - ETA: 0s - loss: 0.2323 - accuracy: 0.9090
Epoch 3: val_accuracy improved from 0.83106 to 0.85963, saving model to /home/hlib/sem7/AI/Lab4_experiment/Results220Neurons/NN_L5_Best.hdf5
102/102 [==============================] - 3s 31ms/step - loss: 0.2322 - accuracy: 0.9088 - val_loss: 0.3463 - val_accuracy: 0.8596
Epoch 4/34
100/102 [============================>.] - ETA: 0s - loss: 0.1797 - accuracy: 0.9333
Epoch 4: val_accuracy did not improve from 0.85963
102/102 [==============================] - 3s 30ms/step - loss: 0.1795 - accuracy: 0.9334 - val_loss: 0.4488 - val_accuracy: 0.8124
Epoch 5/34
101/102 [============================>.] - ETA: 0s - loss: 0.1342 - accuracy: 0.9493
Epoch 5: val_accuracy improved from 0.85963 to 0.87702, saving model to /home/hlib/sem7/AI/Lab4_experiment/Results220Neurons

Epoch 29/34
101/102 [============================>.] - ETA: 0s - loss: 9.8632e-05 - accuracy: 1.0000
Epoch 29: val_accuracy did not improve from 0.92547
102/102 [==============================] - 3s 30ms/step - loss: 9.8738e-05 - accuracy: 1.0000 - val_loss: 0.5110 - val_accuracy: 0.9217
Epoch 30/34
101/102 [============================>.] - ETA: 0s - loss: 9.3212e-05 - accuracy: 1.0000
Epoch 30: val_accuracy did not improve from 0.92547
102/102 [==============================] - 3s 30ms/step - loss: 9.3179e-05 - accuracy: 1.0000 - val_loss: 0.5106 - val_accuracy: 0.9242
Epoch 31/34
101/102 [============================>.] - ETA: 0s - loss: 8.5162e-05 - accuracy: 1.0000
Epoch 31: val_accuracy did not improve from 0.92547
102/102 [==============================] - 3s 30ms/step - loss: 8.4915e-05 - accuracy: 1.0000 - val_loss: 0.5145 - val_accuracy: 0.9230
Epoch 32/34
102/102 [==============================] - ETA: 0s - loss: 7.7569e-05 - accuracy: 1.0000
Epoch 32: val_accuracy did not i

...

102/102 [==============================] - 4s 39ms/step - loss: 0.3323 - accuracy: 0.8653 - val_loss: 0.4407 - val_accuracy: 0.8646
Epoch 3/52
100/102 [============================>.] - ETA: 0s - loss: 0.2428 - accuracy: 0.9062
Epoch 3: val_accuracy did not improve from 0.86460
102/102 [==============================] - 3s 32ms/step - loss: 0.2442 - accuracy: 0.9052 - val_loss: 0.3711 - val_accuracy: 0.8646
Epoch 4/52
101/102 [============================>.] - ETA: 0s - loss: 0.1821 - accuracy: 0.9338
Epoch 4: val_accuracy did not improve from 0.86460
102/102 [==============================] - 4s 36ms/step - loss: 0.1826 - accuracy: 0.9336 - val_loss: 0.3816 - val_accuracy: 0.8298
Epoch 5/52
101/102 [============================>.] - ETA: 0s - loss: 0.1300 - accuracy: 0.9530
Epoch 5: val_accuracy did not improve from 0.86460
102/102 [==============================] - 4s 41ms/step - loss: 0.1304 - accuracy: 0.9527 - val_loss: 0.3906 - val_accuracy: 0.8609
Epoch 6/52
101/102 [===========

Epoch 30/52
101/102 [============================>.] - ETA: 0s - loss: 9.8535e-05 - accuracy: 1.0000
Epoch 30: val_accuracy did not improve from 0.91180
102/102 [==============================] - 4s 38ms/step - loss: 9.9155e-05 - accuracy: 1.0000 - val_loss: 0.5107 - val_accuracy: 0.9043
Epoch 31/52
102/102 [==============================] - ETA: 0s - loss: 9.3397e-05 - accuracy: 1.0000
Epoch 31: val_accuracy did not improve from 0.91180
102/102 [==============================] - 4s 35ms/step - loss: 9.3397e-05 - accuracy: 1.0000 - val_loss: 0.5057 - val_accuracy: 0.9068
Epoch 32/52
101/102 [============================>.] - ETA: 0s - loss: 8.1856e-05 - accuracy: 1.0000
Epoch 32: val_accuracy did not improve from 0.91180
102/102 [==============================] - 4s 36ms/step - loss: 8.2147e-05 - accuracy: 1.0000 - val_loss: 0.5120 - val_accuracy: 0.9056
Epoch 33/52
100/102 [============================>.] - ETA: 0s - loss: 7.1980e-05 - accuracy: 1.0000
Epoch 33: val_accuracy did not i

102/102 [==============================] - 4s 36ms/step - loss: 0.2010 - accuracy: 0.9202 - val_loss: 0.3014 - val_accuracy: 0.8870
Epoch 5/52
101/102 [============================>.] - ETA: 0s - loss: 0.1227 - accuracy: 0.9558
Epoch 5: val_accuracy improved from 0.88696 to 0.91429, saving model to /home/hlib/sem7/AI/Lab4_experiment/Results490Neurons/NN_L5_Best.hdf5
102/102 [==============================] - 4s 40ms/step - loss: 0.1232 - accuracy: 0.9556 - val_loss: 0.2361 - val_accuracy: 0.9143
Epoch 6/52
101/102 [============================>.] - ETA: 0s - loss: 0.0863 - accuracy: 0.9680
Epoch 6: val_accuracy did not improve from 0.91429
102/102 [==============================] - 5s 53ms/step - loss: 0.0865 - accuracy: 0.9678 - val_loss: 0.2726 - val_accuracy: 0.9068
Epoch 7/52
101/102 [============================>.] - ETA: 0s - loss: 0.0545 - accuracy: 0.9817
Epoch 7: val_accuracy did not improve from 0.91429
102/102 [==============================] - 4s 39ms/step - loss: 0.0544 - 

Epoch 32/52
101/102 [============================>.] - ETA: 0s - loss: 7.6538e-05 - accuracy: 1.0000
Epoch 32: val_accuracy did not improve from 0.92547
102/102 [==============================] - 4s 35ms/step - loss: 7.6311e-05 - accuracy: 1.0000 - val_loss: 0.5083 - val_accuracy: 0.9255
Epoch 33/52
100/102 [============================>.] - ETA: 0s - loss: 7.1946e-05 - accuracy: 1.0000
Epoch 33: val_accuracy did not improve from 0.92547
102/102 [==============================] - 4s 35ms/step - loss: 7.2227e-05 - accuracy: 1.0000 - val_loss: 0.5115 - val_accuracy: 0.9255
Epoch 34/52
101/102 [============================>.] - ETA: 0s - loss: 6.2806e-05 - accuracy: 1.0000
Epoch 34: val_accuracy did not improve from 0.92547
102/102 [==============================] - 4s 35ms/step - loss: 6.2664e-05 - accuracy: 1.0000 - val_loss: 0.5112 - val_accuracy: 0.9242
Epoch 35/52
101/102 [============================>.] - ETA: 0s - loss: 5.6770e-05 - accuracy: 1.0000
Epoch 35: val_accuracy did not i

Epoch 33/53
100/102 [============================>.] - ETA: 0s - loss: 6.8933e-05 - accuracy: 1.0000
Epoch 33: val_accuracy did not improve from 0.90932
102/102 [==============================] - 4s 41ms/step - loss: 6.8830e-05 - accuracy: 1.0000 - val_loss: 0.5470 - val_accuracy: 0.9031
Epoch 34/53
100/102 [============================>.] - ETA: 0s - loss: 6.0060e-05 - accuracy: 1.0000
Epoch 34: val_accuracy did not improve from 0.90932
102/102 [==============================] - 4s 40ms/step - loss: 6.0510e-05 - accuracy: 1.0000 - val_loss: 0.5536 - val_accuracy: 0.9056
Epoch 35/53
101/102 [============================>.] - ETA: 0s - loss: 5.6329e-05 - accuracy: 1.0000
Epoch 35: val_accuracy did not improve from 0.90932
102/102 [==============================] - 4s 38ms/step - loss: 5.8529e-05 - accuracy: 1.0000 - val_loss: 0.5565 - val_accuracy: 0.9056
Epoch 36/53
101/102 [============================>.] - ETA: 0s - loss: 5.4912e-05 - accuracy: 1.0000
Epoch 36: val_accuracy did not i

In [ ]:
pip freeze

In [ ]:

#     folder=f"Results15Neurons"
#     folder = 'Lab4_experiment' + "/" + folder

#     print('---start Learn---', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
#     Learn_NN_5L_(TrainDir='/home/hlib/sem7/AI/NewData/Train/', 
#                  ValidDir='/home/hlib/sem7/AI/NewData/Valid/', 
#                  RezDir=f"/home/hlib/sem7/AI/{folder}/", 
#                  NN_Name='NN_L5', Epochs=25, window_size=25, windoe_fuction='hann')

#     print('---end  Learn---', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))

#     for dir in ('Train', 'Test', 'Valid'):
#         TestNN_(NetName=f"/home/hlib/sem7/AI/{folder}/NN_L5_Best.hdf5",
#                 SourceDir=f"/home/hlib/sem7/AI/NewData/{dir}/",
#                 TargetFile=f"/home/hlib/sem7/AI/{folder}/NN_L5_rez_{dir}",
#                 window_size=25)


#     Learn_NN_5L_(TrainDir='/home/hlib/sem7/AI/NewData/Train/', 
#                  ValidDir='/home/hlib/sem7/AI/NewData/Valid/', 
#                  RezDir=f"/home/hlib/sem7/AI/{folder}/", 
#                  NN_Name='NN_L5', Epochs=20+e, window_size=25, windoe_fuction='hann')


#     for dir in ('Train'):
#         TestNN_(NetName=f"/home/hlib/sem7/AI/{folder}/NN_L5_Best.hdf5",
#                 SourceDir=f"/home/hlib/sem7/AI/NewData/{dir}/",
#                 TargetFile=f"/home/hlib/sem7/AI/{d}/NN_L5_rez_{dir}",
#                 window_size=25)

        
        
        
        
        
        
#     TestNN_Custom_2(NetName=f"/home/hlib/sem7/AI/{folder}/NN_L5_Best.hdf5",
#         SourceDir=f"/home/hlib/sem7/AI/NewData/",
#         TargetFolder=f"/home/hlib/sem7/AI/{folder}/",
#         window_size=25)
    
    
    
    
    
#     for dir in ('Train', 'Test', 'Valid'):
#         TestNN_(NetName=f"/home/hlib/sem7/AI/{folder}/NN_L5_Best.hdf5",
#                 SourceDir=f"/home/hlib/sem7/AI/NewData/{dir}/",
#                 TargetFile=f"/home/hlib/sem7/AI/{d}/NN_L5_rez_{dir}",
#                 window_size=25)
    
    # print('---end  Learn---', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
#     print(f"start test for {neuro} neurons")
#     try: 
#     TestNN_Custom_2(NetName=f"/home/hlib/sem7/AI/{folder}/NN_L5_Best.hdf5",
#         SourceDir=f"/home/hlib/sem7/AI/NewData/",
#         TargetFolder=f"/home/hlib/sem7/AI/{d}/",
#         window_size=25)
#     except:
#         print("got error")


In [9]:
def Learn_NN_5L_Custom_2(TrainDir,ValidDir, RezDir,NN_Name, Neurons=40, Epochs=30, window_size=25, windoe_fuction='hann', optimizerFunc=Adam()):
    Source_Samples, Input_Files= Load_Wav_(TrainDir)
    print('end load train data', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
    Source_Samples, Input_Files = RandomozeArrays(Source_Samples, Input_Files)
    print('end Randomize train data', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
    
    
    X_Train, Y_Train = Convert_Wav_To_specgram(SamplesList=Source_Samples, Input_Files=Input_Files,
                                           window_size=window_size, windoe_fuction=windoe_fuction)
    print('end convert train data', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
     
    Source_Samples, Input_Files = Load_Wav_(ValidDir)
    print('end load valid data', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))

    X_val1, y_val1 = Convert_Wav_To_specgram(SamplesList=Source_Samples, Input_Files=Input_Files,
                                                     window_size=window_size, windoe_fuction=windoe_fuction)
    print('end convert valid data', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))

    input_shape = (X_Train.shape[1], X_Train.shape[2], 1)
    model = Sequential()

    model.add(BatchNormalization(input_shape = input_shape))
    model.add(Convolution2D(48, (5, 5), strides = (3, 3), padding = 'same',input_shape = input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding = 'same'))
    model.add(BatchNormalization())
    model.add(Convolution2D(40, (3, 3), strides = (2, 2), padding = 'same'))
    model.add(Activation('relu'))
    model.add(Flatten())
#     model.add(Dense(45))
#     model.add(Activation('relu'))
#     model.add(Dense(15))
#     model.add(Activation('relu'))
#     model.add(Dense(10))
#     model.add(Activation('relu'))
    model.add(Dense(Neurons))
    model.add(Activation('relu'))
    model.add(Dense(3))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizerFunc, metrics=['accuracy'])
    csv_logger = CSVLogger(RezDir+NN_Name+'_training__log.csv', separator=',', append=False)

    
    checkpoint = ModelCheckpoint(filepath=RezDir+NN_Name+'_Best.hdf5',
                 monitor='val_accuracy',
                 save_best_only=True,
                 mode='max',
                 verbose=1)
    model.fit(X_Train, Y_Train,
          batch_size = 64,
          epochs = Epochs,shuffle=True,
          validation_data=(X_val1, y_val1),
          callbacks=[checkpoint, csv_logger])
    model.save(filepath=RezDir+NN_Name+'_Final.hdf5')

In [11]:
d = 'Lab5_experiment'
# stopped at Nadam 105
# for neuro in [15]: 
# for neuro in range (5, 201, 5):
# for optimizer, optimizerFunc in zip(['Nadam'], [Nadam]):
for optimizer, optimizerFunc in zip(['Ftrl'], [Ftrl]):
#for optimizer, optimizerFunc in zip(['Adam', 'Adagrad', 'SGD', 'RMSprop', 'Adadelta', 'Adamax', 'Nadam', 'Ftrl'], [Adam, Adagrad, SGD, RMSprop, Adadelta, Adamax, Nadam, Ftrl]):
#     for neuro in range (130, 400, 25): 
    for neuro in range (30, 400, 25):
        folder=f"Results{neuro}Neurons_{optimizer}"
        folder = d + "/" + folder

        e = neuro // 15

        print(f"start train for {neuro} neurons, {optimizer}, optimizer")

        Learn_NN_5L_Custom_2(TrainDir='/home/hlib/sem7/AI/NewData/Train/', 
                     ValidDir='/home/hlib/sem7/AI/NewData/Valid/', 
                     RezDir=f"/home/hlib/sem7/AI/{folder}/", 
                     NN_Name='NN_L5', Neurons=neuro, Epochs=20+e, window_size=25, windoe_fuction='hann',optimizerFunc=optimizerFunc())


        NetName = f"/home/hlib/sem7/AI/{folder}/NN_L5_Best.hdf5"
        TargetFile = f"/home/hlib/sem7/AI/{d}/"

        f = open(TargetFile+'Report.csv', 'a', newline='\n')
        f.write('NetName = %s,'%(NetName))
        f.write(f"{neuro},{optimizer},")
        f.close()

        for dir in ('Train', 'Test', 'Valid'):
            TestNN_Custom_2(NetName=NetName,
                    SourceDir=f"/home/hlib/sem7/AI/NewData/{dir}/",
                    TargetFile=TargetFile,
                    window_size=25)

        f = open(TargetFile+'Report.csv', 'a', newline='\n')
        f.write('\n')
        f.close()   

        print(f"finish train for {neuro} neurons, {optimizer}, optimizer")

start train for 30 neurons, Ftrl, optimizer
end load train data 2022-11-14 20:23:14
end Randomize train data 2022-11-14 20:23:14
end convert train data 2022-11-14 20:23:16
end load valid data 2022-11-14 20:23:16
end convert valid data 2022-11-14 20:23:16
Epoch 1/22
102/102 [==============================] - ETA: 0s - loss: 1.0951 - accuracy: 0.5401
Epoch 1: val_accuracy improved from -inf to 0.33540, saving model to /home/hlib/sem7/AI/Lab5_experiment/Results30Neurons_Ftrl/NN_L5_Best.hdf5
102/102 [==============================] - 4s 36ms/step - loss: 1.0951 - accuracy: 0.5401 - val_loss: 1.0987 - val_accuracy: 0.3354
Epoch 2/22
102/102 [==============================] - ETA: 0s - loss: 1.0908 - accuracy: 0.5419
Epoch 2: val_accuracy did not improve from 0.33540
102/102 [==============================] - 3s 25ms/step - loss: 1.0908 - accuracy: 0.5419 - val_loss: 1.0987 - val_accuracy: 0.3354
Epoch 3/22
100/102 [============================>.] - ETA: 0s - loss: 1.0882 - accuracy: 0.5419


101/102 [============================>.] - ETA: 0s - loss: 1.0790 - accuracy: 0.5418
Epoch 9: val_accuracy did not improve from 0.33540
102/102 [==============================] - 3s 28ms/step - loss: 1.0790 - accuracy: 0.5419 - val_loss: 1.0995 - val_accuracy: 0.3354
Epoch 10/25
102/102 [==============================] - ETA: 0s - loss: 1.0779 - accuracy: 0.5419
Epoch 10: val_accuracy did not improve from 0.33540
102/102 [==============================] - 3s 28ms/step - loss: 1.0779 - accuracy: 0.5419 - val_loss: 1.0996 - val_accuracy: 0.3354
Epoch 11/25
100/102 [============================>.] - ETA: 0s - loss: 1.0768 - accuracy: 0.5422
Epoch 11: val_accuracy did not improve from 0.33540
102/102 [==============================] - 3s 28ms/step - loss: 1.0769 - accuracy: 0.5419 - val_loss: 1.0998 - val_accuracy: 0.3354
Epoch 12/25
101/102 [============================>.] - ETA: 0s - loss: 1.0759 - accuracy: 0.5416
Epoch 12: val_accuracy did not improve from 0.33540
102/102 [============

101/102 [============================>.] - ETA: 0s - loss: 1.0772 - accuracy: 0.5410
Epoch 11: val_accuracy did not improve from 0.33540
102/102 [==============================] - 3s 28ms/step - loss: 1.0771 - accuracy: 0.5419 - val_loss: 1.0997 - val_accuracy: 0.3354
Epoch 12/28
101/102 [============================>.] - ETA: 0s - loss: 1.0762 - accuracy: 0.5415
Epoch 12: val_accuracy did not improve from 0.33540
102/102 [==============================] - 3s 28ms/step - loss: 1.0761 - accuracy: 0.5419 - val_loss: 1.0998 - val_accuracy: 0.3354
Epoch 13/28
101/102 [============================>.] - ETA: 0s - loss: 1.0752 - accuracy: 0.5416
Epoch 13: val_accuracy did not improve from 0.33540
102/102 [==============================] - 3s 28ms/step - loss: 1.0752 - accuracy: 0.5419 - val_loss: 1.1000 - val_accuracy: 0.3354
Epoch 14/28
100/102 [============================>.] - ETA: 0s - loss: 1.0745 - accuracy: 0.5402
Epoch 14: val_accuracy did not improve from 0.33540
102/102 [===========

100/102 [============================>.] - ETA: 0s - loss: 1.0810 - accuracy: 0.5406
Epoch 7: val_accuracy did not improve from 0.33540
102/102 [==============================] - 3s 29ms/step - loss: 1.0809 - accuracy: 0.5419 - val_loss: 1.0994 - val_accuracy: 0.3354
Epoch 8/32
101/102 [============================>.] - ETA: 0s - loss: 1.0795 - accuracy: 0.5419
Epoch 8: val_accuracy did not improve from 0.33540
102/102 [==============================] - 3s 28ms/step - loss: 1.0795 - accuracy: 0.5419 - val_loss: 1.0995 - val_accuracy: 0.3354
Epoch 9/32
100/102 [============================>.] - ETA: 0s - loss: 1.0783 - accuracy: 0.5414
Epoch 9: val_accuracy did not improve from 0.33540
102/102 [==============================] - 3s 28ms/step - loss: 1.0782 - accuracy: 0.5419 - val_loss: 1.0996 - val_accuracy: 0.3354
Epoch 10/32
101/102 [============================>.] - ETA: 0s - loss: 1.0769 - accuracy: 0.5422
Epoch 10: val_accuracy did not improve from 0.33540
102/102 [================

102/102 [==============================] - ETA: 0s - loss: 1.0561 - accuracy: 0.5419
Epoch 31: val_accuracy did not improve from 0.33540
102/102 [==============================] - 2s 24ms/step - loss: 1.0561 - accuracy: 0.5419 - val_loss: 1.1041 - val_accuracy: 0.3354
Epoch 32/33
100/102 [============================>.] - ETA: 0s - loss: 1.0547 - accuracy: 0.5428
Epoch 32: val_accuracy did not improve from 0.33540
102/102 [==============================] - 2s 24ms/step - loss: 1.0549 - accuracy: 0.5419 - val_loss: 1.1044 - val_accuracy: 0.3354
Epoch 33/33
101/102 [============================>.] - ETA: 0s - loss: 1.0537 - accuracy: 0.5425
Epoch 33: val_accuracy did not improve from 0.33540
26/26 [==============================] - 0s 3ms/step
finish train for 205 neurons, Ftrl, optimizer
start train for 230 neurons, Ftrl, optimizer
end load train data 2022-11-14 20:35:01
end Randomize train data 2022-11-14 20:35:01
end convert train data 2022-11-14 20:35:02
end load valid data 2022-11-1

102/102 [==============================] - ETA: 0s - loss: 1.0678 - accuracy: 0.5419
Epoch 17: val_accuracy did not improve from 0.33540
102/102 [==============================] - 3s 25ms/step - loss: 1.0678 - accuracy: 0.5419 - val_loss: 1.1012 - val_accuracy: 0.3354
Epoch 18/37
102/102 [==============================] - ETA: 0s - loss: 1.0667 - accuracy: 0.5419
Epoch 18: val_accuracy did not improve from 0.33540
102/102 [==============================] - 3s 25ms/step - loss: 1.0667 - accuracy: 0.5419 - val_loss: 1.1014 - val_accuracy: 0.3354
Epoch 19/37
102/102 [==============================] - ETA: 0s - loss: 1.0655 - accuracy: 0.5419
Epoch 19: val_accuracy did not improve from 0.33540
102/102 [==============================] - 2s 24ms/step - loss: 1.0655 - accuracy: 0.5419 - val_loss: 1.1016 - val_accuracy: 0.3354
Epoch 20/37
100/102 [============================>.] - ETA: 0s - loss: 1.0642 - accuracy: 0.5428
Epoch 20: val_accuracy did not improve from 0.33540
102/102 [===========

101/102 [============================>.] - ETA: 0s - loss: 1.0567 - accuracy: 0.5421
Epoch 36: val_accuracy did not improve from 0.33540
102/102 [==============================] - 4s 35ms/step - loss: 1.0567 - accuracy: 0.5419 - val_loss: 1.1038 - val_accuracy: 0.3354
Epoch 37/38
100/102 [============================>.] - ETA: 0s - loss: 1.0559 - accuracy: 0.5414
Epoch 37: val_accuracy did not improve from 0.33540
102/102 [==============================] - 3s 33ms/step - loss: 1.0558 - accuracy: 0.5419 - val_loss: 1.1041 - val_accuracy: 0.3354
Epoch 38/38
101/102 [============================>.] - ETA: 0s - loss: 1.0549 - accuracy: 0.5418
Epoch 38: val_accuracy did not improve from 0.33540
26/26 [==============================] - 0s 3ms/step
finish train for 280 neurons, Ftrl, optimizer
start train for 305 neurons, Ftrl, optimizer
end load train data 2022-11-14 20:40:22
end Randomize train data 2022-11-14 20:40:22
end convert train data 2022-11-14 20:40:23
end load valid data 2022-11-1

101/102 [============================>.] - ETA: 0s - loss: 1.0757 - accuracy: 0.5424
Epoch 12: val_accuracy did not improve from 0.33540
102/102 [==============================] - 2s 24ms/step - loss: 1.0757 - accuracy: 0.5419 - val_loss: 1.0999 - val_accuracy: 0.3354
Epoch 13/42
100/102 [============================>.] - ETA: 0s - loss: 1.0747 - accuracy: 0.5422
Epoch 13: val_accuracy did not improve from 0.33540
102/102 [==============================] - 3s 25ms/step - loss: 1.0748 - accuracy: 0.5419 - val_loss: 1.1000 - val_accuracy: 0.3354
Epoch 14/42
102/102 [==============================] - ETA: 0s - loss: 1.0738 - accuracy: 0.5419
Epoch 14: val_accuracy did not improve from 0.33540
102/102 [==============================] - 3s 25ms/step - loss: 1.0738 - accuracy: 0.5419 - val_loss: 1.1001 - val_accuracy: 0.3354
Epoch 15/42
100/102 [============================>.] - ETA: 0s - loss: 1.0730 - accuracy: 0.5419
Epoch 15: val_accuracy did not improve from 0.33540
102/102 [===========

101/102 [============================>.] - ETA: 0s - loss: 1.0651 - accuracy: 0.5416
Epoch 26: val_accuracy did not improve from 0.33540
102/102 [==============================] - 4s 43ms/step - loss: 1.0650 - accuracy: 0.5419 - val_loss: 1.1017 - val_accuracy: 0.3354
Epoch 27/43
101/102 [============================>.] - ETA: 0s - loss: 1.0643 - accuracy: 0.5421
Epoch 27: val_accuracy did not improve from 0.33540
102/102 [==============================] - 3s 31ms/step - loss: 1.0644 - accuracy: 0.5419 - val_loss: 1.1018 - val_accuracy: 0.3354
Epoch 28/43
102/102 [==============================] - ETA: 0s - loss: 1.0637 - accuracy: 0.5419
Epoch 28: val_accuracy did not improve from 0.33540
102/102 [==============================] - 4s 34ms/step - loss: 1.0637 - accuracy: 0.5419 - val_loss: 1.1020 - val_accuracy: 0.3354
Epoch 29/43
102/102 [==============================] - ETA: 0s - loss: 1.0631 - accuracy: 0.5419
Epoch 29: val_accuracy did not improve from 0.33540
102/102 [===========

101/102 [============================>.] - ETA: 0s - loss: 1.0334 - accuracy: 0.5413
Epoch 39: val_accuracy did not improve from 0.33540
102/102 [==============================] - 3s 30ms/step - loss: 1.0332 - accuracy: 0.5419 - val_loss: 1.1151 - val_accuracy: 0.3354
Epoch 40/45
101/102 [============================>.] - ETA: 0s - loss: 1.0312 - accuracy: 0.5418
Epoch 40: val_accuracy did not improve from 0.33540
102/102 [==============================] - 3s 31ms/step - loss: 1.0311 - accuracy: 0.5419 - val_loss: 1.1166 - val_accuracy: 0.3354
Epoch 41/45
100/102 [============================>.] - ETA: 0s - loss: 1.0287 - accuracy: 0.5427
Epoch 41: val_accuracy did not improve from 0.33540
102/102 [==============================] - 3s 31ms/step - loss: 1.0290 - accuracy: 0.5419 - val_loss: 1.1183 - val_accuracy: 0.3354
Epoch 42/45
101/102 [============================>.] - ETA: 0s - loss: 1.0273 - accuracy: 0.5410
Epoch 42: val_accuracy did not improve from 0.33540
102/102 [===========